In [18]:
from collections import deque

In [146]:
%%file breadth_first_search.py
'''module implements a breadth-first search algorithm, computes the set of
connected components of an undirected graph and determines the size of the
largest connected component. Also implements a function that computes the
resilience of a graph as nodes are deleted from it.'''

from collections import deque

def bfs_visited(ugraph, start_node):
    '''takes the provided undirected graph ugraph and the node start_node
    and returns the set consisting of all nodes that are visited by a 
    breadth-first search that starts at start_node'''
    queue = deque()
    visited = {start_node}
    queue.append(start_node)
    while queue:
        next_node = queue.popleft()
        for neighbor in ugraph[next_node]:
            if neighbor not in visited:
                visited.update({neighbor})
                queue.append(neighbor)
    return visited

def cc_visited(ugraph):
    '''takes the undirected graph ugraph and returns a list of sets, where
    each set consists of all the nodes in a connected component. There is one
    set in the list for each connected component in ugraph and nothing else'''
    remaining_nodes = list(ugraph.keys())
    connected_components = []
    while remaining_nodes:
        node = remaining_nodes[0]
        visited = bfs_visited(ugraph, node)
        connected_components.append(visited)
        for vnode in visited:
            remaining_nodes.remove(vnode)
    return connected_components
    
def largest_cc_size(ugraph):
    '''takes the undirected graph ugraph and returns the size of the largest
    connected component in ugraph'''
    connected_components = cc_visited(ugraph)
    if not connected_components:
        return 0
    return max([len(component) for component in connected_components])

def compute_resilience(ugraph, attack_order):
    '''takes the undirected graph ugraph and a list of nodes attack_order and
    iterates through the nodes in attack_order. For each node in the list, the
    function removes the given node and its edges from the graph and then
    computes the size of the largest connected compontent for the resulting
    graph. Returns a list whose k+1 entry in the size of the largest connected
    component in the resulting graphs after the removal of the first k nodes
    in attack_order'''
    attack_result = [largest_cc_size(ugraph)]
    for node in attack_order:
        for neighbor in ugraph[node]:
            ugraph[neighbor].remove(node)
        del(ugraph[node])
        attack_result.append(largest_cc_size(ugraph))
    return attack_result

Overwriting breadth_first_search.py


In [145]:
GRAPH0 = {0: set([1]),
          1: set([0, 2]),
          2: set([1, 3]),
          3: set([2]),
          4: set([5]),
          5: set([4])}

assert(bfs_visited(GRAPH0, 0) == {0, 1, 2, 3})

assert(bfs_visited(GRAPH0, 4) == {4, 5})

assert(cc_visited(GRAPH0) == [{0, 1, 2, 3}, {4, 5}])

assert(largest_cc_size(GRAPH0) == 4)

assert(compute_resilience(GRAPH0, [0, 1, 2, 3, 4, 5]) == [4, 3, 2, 2, 2, 1, 0])